<div style="
    font-weight: bold; 
    font-size: 28px;
    ">
    [Notebook Title]
</div>

<br>

<div style="font-size:15px; line-height:1.8">
    
**Name**: [Enter your name here] <br>
**Email**: [Enter your email here] <br>

</div>

---

<div style="font-size:15px; line-height:1.8;">

**Description**

...

**Table of Content**

<ol>
    <li style="margin-bottom: 8px; font-weight: bold;">Exploratory Data Analysis (EDA)</li>
    <li style="margin-bottom: 8px; font-weight: bold;">Preprocessing Steps</li>
    <p>
        A. Feature Engineering <br> 
        B. Handle Missing Values <br>
        C. Handle Noisy Data <br>
        D. Handle Categorical Variables <br>
    </p>
    <li style="margin-bottom: 8px; font-weight: bold;">Model Building & Comparison</li>
    <p>
        A. Decision Trees <br> 
        B. Random Forests <br>
        C. XGBoost <br>
        D. Compare Models <br>
    </p>
    <li style="margin-bottom: 8px; font-weight: bold;">Pipeline Building</li>
</ol>

</div>

---

In [ ]:
# If using dark mode, execute the following:
# import matplotlib.pyplot as plt

# plt.rcParams.update(
#     {
#         "figure.facecolor": "#111111",
#         "text.color": "white",
#         "xtick.color": "white",
#         "ytick.color": "white",
#         "axes.labelcolor": "white",
#         "axes.edgecolor": "white",
#         "axes.facecolor": "#111111",
#     }
# )

# Reset changes:
# plt.rcdefaults()

<h2 style="font-weight: bold; font-size: 22px;">[Section Heading]</h2>

---

...

<h3 style="font-weight: bold; font-size: 18px;">[Mini Section Heading]</h3>

<div style="font-size:15px; line-height:1.8;">

Normal Text

</div>

<code style='font-size:14px; font-weight:bold;'>code</code>

<div style="font-size:15px; line-height:1.8;">

<ul>
    <li style="margin-bottom: 8px; font-weight: bold;">...</li>
    <p>
        - ... <br> 
        - ...
    </p>
    <li style="margin-bottom: 8px; font-weight: bold;">...</li>
    <p>
        - ... <br>
        - ...
    </p>
</ul>

</div>

<div style="font-size:15px; line-height: 1.8;">

<table>
    <tr>
        <th>Column 1</th>
        <th>Column 2</th>
    </tr>
    <tbody>
        <tr>
            <td>Row 1, Column 1</td>
            <td>Row 1, Column 2</td>
        </tr>
        <tr>
            <td>Row 2, Column 1</td>
            <td>Row 2, Column 2</td>
        </tr>      
    </tbody>
</table>
    
</div>

<div style="font-size:15px; line-height: 1.8;">

<blockquote>
...
</blockquote>
<figcaption>Source, <cite>Link</cite></figcaption>

</div>